<a href="https://colab.research.google.com/github/ahmeda335/BurgerResturantSimulation_Outlines/blob/main/BurgerResturant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Making an LLM response for customers' requests in a Burger resturant.***🩻♻️🍔
## - This code tries to mimitate the jop of a phone order taker in a burger resturant, by using a model that take orders from customers in natural language then tries to extract the components of the order, then shaping it in a json format.<br>
---
# Note:
## --> This is not an optimal code, it handles some of cases, but there still many cases to handle more 😊.
## --> There are other libraries that offers the same advantage - and maybe better -.
## --> The efficiency of the code depends on the model used.

###⚡ Installing required packages.

In [ ]:
!pip install -q outlines
!pip install -q transformers
!pip install -q autoawq
!pip install -q accelerate
!pip install -q torch==2.4.0

In [ ]:
import outlines
import torch

from enum import Enum
from pydantic import BaseModel

### ⚡ Setting outlines prompts used in the code.

##### Prompt of taking orders from customers.

In [ ]:
@outlines.prompt
def take_order(order):
    """You are the owner of a Burger Resturant. Customers \
    send you orders from which you need to extract:

    1. The Burger that is ordered
    2. The number of Burgers
    3. The drink that is ordered if any.
    4. The number of the drinks if any.
    5. The addings that are ordered if any.

    # EXAMPLE

    ORDER: I would like two Cheese Burger with Onion and Lettuce, and one Coke.
    RESULT: {"Burger": "Cheese Burger", "number": 2, "Drink": "Coke", "Addings": "Onion, Lettuce"}


    # OUTPUT INSTRUCTIONS

    Answer in valid JSON. Here are the different objects relevant for the output:

    Order:
        Burger (str): name of the burger
        burger_number (int): number of Burgars
        Drink (str): name of the drink
        drink_number (int): number of drinks
        Addings (list[str]): list of addings


    Return a valid JSON of type "Order"

    # OUTPUT

    ORDER: {{ order }}
    RESULT: """

##### Prompt of verifing that the order is correct.

In [ ]:
@outlines.prompt
def order_verify(answer):
    """You are the owner of a Burger Resturant, and customers \
    send you orders then you extracted what they ordered, then you \
    need to verify if they are correct.


    # EXAMPLE

    ORDER: I would like two Cheese Burger with Onion and Lettuce, and one Coke.
    RESULT: {"Burger": "Cheese Burger", "number": 2, "Drink": "Coke", "Addings": "Onion, Lettuce"}
    VERIFICATION: The order is correct.
    Customer: Yes the order is correct.
    return: True [boolean value]


    # OUTPUT INSTRUCTIONS

    Answer in a boolean value. Here are the different objects relevant for the output:

    makingSure:
        True or False (boolean value)


    Return a boolean value

    # OUTPUT

    Verification: {{ answer }}
    RESULT: """

###  ⚡Setting some classes used in the prompts using pydantic to detect the shape of the out.

In [ ]:
class Burger(str, Enum):
    Smash_Burger = "Smash Burger"
    Cheese_Burger = "Cheese Burger"
    Double_Cheese_Burger = "Double Cheese Burger"
    Chicken_Burger = "Chicken Burger"
    Veggie_Burger = "Veggie Burger"


class Drink(str, Enum):
    Coke = "Coke"
    Pepsi = "Pepsi"
    Sprite = "Sprite"
    Fanta = "Fanta"
    Water = "Water"


class Addings(str, Enum):
    Lettuce = "Lettuce"
    Tomato = "Tomato"
    Onion = "Onion"
    Pickle = "Pickle"
    Mushroom = "Mushroom"
    Cheese = "Cheese"
    Mayo = "Mayo"
    Mustard = "Mustard"
    Ketchup = "Ketchup"
    BBQ_Sauce = "BBQ Sauce"

class BurgerOrder(BaseModel):
    burger: Burger
    number: int
    addings: list[Addings]

class DrinkOrder(BaseModel):
    drink: Drink
    number: int

class OneOrder(BaseModel):
    burger: list[BurgerOrder]
    drink: list[DrinkOrder]
    order: int

# class Orders(BaseModel):
#     order: OneOrder
#     number: int


class Verify(BaseModel):
    makingSure: bool

###  ⚡ Setting the model used.

In [ ]:
model = outlines.models.transformers("TheBloke/Mistral-7B-OpenOrca-AWQ", device="cuda")

### ⚡ Taking the order from the customer and verify it.

In [ ]:
# The customer enter the order here.
orders = ["Hi! I want one cheese burger with no addings, two smash burgers with lettuce and tomato, two coke, and a bottle of water."]


def taking_order(_orders):

  order_prompts = take_order(_orders)
  order_generator = outlines.generate.json(model, OneOrder)
  results = order_generator(order_prompts)
  return results


In [ ]:
results = taking_order(orders)
print(results)

In [ ]:
def orders_verify(results):
    burgers = results.burger
    drinks = results.drink

    print("You ordered:")
    for burger in burgers:
      print(f"- {burger.number} {burger.burger} with {', '.join([adding.value for adding in burger.addings])}")
    for drink in drinks:
      print(f"- {drink.number} {drink.drink}")
    print("Are these orders correct? ")
    ans = "No, My orders are not correct"  # "No these orders are not correct."

    verify_prompts = order_verify(ans)
    verify_generator = outlines.generate.json(model, Verify)
    makingSure = verify_generator(verify_prompts)
    print (makingSure)

    verify = makingSure.makingSure

    return verify

In [ ]:
def setting_order(results):
  print("Your order is on the way....")
  print("Thank you for your order!")

In [ ]:
verify = False

# As long as the order is not correct, the customer enter his order again.
while not verify:
  verify = orders_verify(results)
  if verify:
    setting_order(results)
    continue
  else:
    print("Please enter your orders again")
    orders = input("Type your orders here: ")
    results = taking_order(orders)